# Table of Contents
## 1. [Data-Preprocessing](#Data-Preprocessing)
## 2. [Porfolio](#Portfolio**)
## 3. [Clustering](#Clustering)
## ?. [Main Script](#Main-script)

# `src/preprocessing-utils.R` $\rightarrow$ `src/preprocessing_utils.py`

In [1]:
import pandas as pd
import numpy as np
import os
from functools import reduce

In [2]:
def as_quarter(months):
    """
    월(month) 값을 분기(quarter) 값으로 전환
    Example: (1, 2, 3, 4, 5, 6) -> (1, 1, 1, 2, 2, 2)
    
    INPUT:
      x: 1 ~ 12 사이의 integer vector
    RETURN:
      월 -> 분기로 전환된 integer vector
    """
    months = pd.to_numeric(months)
    if not np.all(np.isin(months, np.arange(1, 13))):
        raise ValueError("range of months exceeds 1~12")
    return (np.array(months) - 1) // 3 + 1

In [3]:
def is_quarter_interval(date_str):
    """
    시간 변수가 날짜 형식(ex: "20010131")인지 
    혹은 분기 형식(ex: "2001/1 Quarter)인지 검사
    
    INPUT:
      x: date string vector
    RETURN:
      날짜 형식이면 FALSE, 분기 형식이면 TRUE
    """
    return len(str(date_str)) == 5

In [4]:
def reshape_long(df):
    """
    Short form 데이터를 long form 데이터로 변환
    시간 변수가 날짜 형식인 경우 분기 형식으로 변환
    
    INPUT:
      data: raw data를 불러들인 data frame
    RETURN:
      long form으로 전환된 data frame
    """
    df = gather(df)
    df.val = pd.to_numeric(df.val)
    df = tidyup_timeframe(df, is_quarter_interval(df.time[0]))
    return df

In [5]:
def gather(df):
    df.rename(index=str, columns={"Unnamed: 1": "code", "Unnamed: 2": "name"}, inplace=True)
    df.columns = ["y"+format_quarter(str(c)) if c[0].isnumeric() else c for c in df.columns]
    df = pd.wide_to_long(df, stubnames="y", i=["code", "name"], j="time")
    df = pd.DataFrame(df.to_records()).rename(index=str, columns={"y": "val"})
    return df

In [6]:
def format_quarter(string):
    return string.replace(" ", "").replace("/", "").replace("Quarter", "").replace("SemiAnnual", "2").replace("Annual", "4")

In [7]:
def tidyup_timeframe(df, is_quarter):
    df.val = pd.to_numeric(df.val)
    if is_quarter:
        year_quarter = df.time.astype(str).str.extract('(.{4,4})(.{1,1})')
        year_quarter.columns = ["year", "quarter"]
        yq = year_quarter.year + "-" + year_quarter.quarter
        yq.name = "time"
        df = pd.concat([df.iloc[:, :2], yq, df.val], axis=1)
    else:
        year_quarter = df.time.astype(str).str.extract('(.{4,4})(.{2,2})')
        year_quarter.columns = ["year", "quarter"]
        yq = year_quarter.year + "-" + as_quarter(year_quarter.quarter).astype(str)
        yq.name = "time"
        df = pd.concat([df.iloc[:, :2], yq, df.val], axis=1)
        df = pd.DataFrame(df.groupby(["code", "name", "time"]).mean().to_records())
    df.sort_values(by=["code", "time"], inplace=True)
    return df

In [8]:
def preprocess(path, file_names, var_names, extension=".xls"):
    dfs = []
    for name in file_names:
        print(name, end=", ")
        file_path = os.path.join(path, name+extension)
        data = reshape_long( pd.read_excel(file_path, skiprows=5).iloc[1:, 1:] )
        dfs.append(data)

    vals = reduce(lambda x, y: x.merge(y, how="left", on=["code", "name", "time"]), dfs).iloc[:, 3:]
    vals.columns = var_names
    
    features = extract_features(vals)
    df = pd.concat([data.loc[:, ["code", "time"]], features], axis=1)
    return df

In [9]:
def extract_features(vals):
    leverage = vals.leverage
    asset_growth = vals.asset_growth
    shares_turnover = vals.trade_amount / vals.stock_num
    roa = vals.net_profit / vals.asset
    roe = vals.net_profit / vals.equity
    size = vals.market_cap
    pcr = vals.pcr
    per = vals.per
    equity_turnover = vals.equity_turnover
    volatility = vals.volatility
    logret = np.log(vals.price).diff()
    
    features = pd.concat([leverage, asset_growth, shares_turnover, roa, roe, size, 
                          pcr, per, equity_turnover, volatility, logret], axis=1)
    features.columns = ["leverage", "asset_growth", "shares_turnover", "roa", "roe", "size", 
                        "pcr", "per", "equity_turnover", "volatility", "logret"]
    return features

---

# Data Preprocessing
## `scripts/01_data-preprocessing.R` $\rightarrow$ `scripts/01_data-preprocessing.py`

In [1]:
from src.preprocessing_utils import *

## Stock data

In [2]:
path = os.path.join("data", "raw")
file_names = ["asset", "asset-growth", "equity", "equity-turnover",
              "leverage", "market-cap", "net-profit", "pcr", "per",
              "stock-number", "stock-price", "trade-amount", "volatility"]
var_names = ["asset", "asset_growth", "equity", "equity_turnover",
             "leverage", "market_cap", "net_profit", "pcr", "per",
             "stock_num", "price", "trade_amount", "volatility"]
extension = ".xls"

In [3]:
stock_tbl = preprocess(path, file_names, var_names, extension=".xls")
stock_tbl.head()

asset, asset-growth, equity, equity-turnover, leverage, market-cap, net-profit, pcr, per, stock-number, stock-price, trade-amount, volatility, 

,code,time,leverage,asset_growth,shares_turnover,roa,roe,size,pcr,per,equity_turnover,volatility,logret
0,000020,1997-1,NaN,NaN,0.006314,NaN,NaN,8.850400e+10,NaN,15.586667,NaN,60.255281,NaN
1,000020,1997-2,206.92,23.60,0.015739,0.002507,0.007696,1.104880e+11,NaN,11.053333,2.22,62.004616,0.235505
2,000020,1997-3,NaN,NaN,0.004035,NaN,NaN,1.087867e+11,NaN,10.543333,NaN,56.334550,-0.040738
3,000020,1997-4,324.23,12.41,0.004633,0.009214,0.039088,4.613333e+10,NaN,4.346667,2.89,62.702979,-0.847219
4,000020,1998-1,NaN,NaN,0.012191,NaN,NaN,5.973333e+10,NaN,4.750000,NaN,73.726084,0.120535


## KOSPI index

In [4]:
def unpack_df(df):
    df = df.reset_index()
    df["time"] = df[['time', 'level_1']].astype(str).apply(lambda x: '-'.join(x), axis=1)
    df = pd.concat([df.time, df.price], axis=1)
    df.columns = ["time", "logret"]
    return df

In [14]:
kospi = pd.read_excel("data/raw/kospi-index.xlsx", names=["time", "price"])
g = kospi.groupby([pd.DatetimeIndex(kospi.time).year, as_quarter(pd.DatetimeIndex(kospi.time).month)])
g = np.log(g.mean()).diff()
kospi = unpack_df(g)
kospi.head()

,time,logret
0,1997-1,NaN
1,1997-2,0.082477
2,1997-3,-0.064037
3,1997-4,-0.499768
4,1998-1,0.247514


## Risk-free rate

In [10]:
risk_free = pd.read_excel("data/raw/cd-risk-free.xlsx", names=["time", "r"])
risk_free.time = risk_free.time.str.replace("/", "-").str.split().str[0]
risk_free.r = np.log(1 + risk_free.r / 100)
risk_free.head()

,time,r
0,1997-1,0.119707
1,1997-2,0.118287
2,1997-3,0.119115
3,1997-4,0.145110
4,1998-1,0.204599


## Save processed data

In [11]:
stock_tbl.to_csv("data/processed/stock2.csv", index=False)
kospi.to_csv("data/processed/kospi2.csv", index=False)
risk_free.to_csv("data/processed/risk_free2.csv", index=False)

---

# Portfolio**
## `src/functions-portfolio.R` $\rightarrow$ `src/portfolio.py`

In [240]:
from quadprog import solve_qp

In [241]:
help(solve_qp)

Help on built-in function solve_qp in module quadprog:

solve_qp(...)
    Solve a strictly convex quadratic program
    
    Minimize     1/2 x^T G x - a^T x
    Subject to   C.T x >= b
    
    This routine uses the the Goldfarb/Idnani dual algorithm [1].
    
    References
    ---------
    ... [1] D. Goldfarb and A. Idnani (1983). A numerically stable dual
        method for solving strictly convex quadratic programs.
        Mathematical Programming, 27, 1-33.
    
    Parameters
    ----------
    G : array, shape=(n, n)
        matrix appearing in the quadratic function to be minimized
    a : array, shape=(n,)
        vector appearing in the quadratic function to be minimized
    C : array, shape=(n, m)
        matrix defining the constraints under which we want to minimize the
        quadratic function
    b : array, shape=(m), default=None
        vector defining the constraints
    meq : int, default=0
        the first meq constraints are treated as equality constraints,
 

* `Python`: Minimize $1/2 x^T G x - a^T x$, Subject to $C^T x >= b$.
* `R`: $min(−d^Tb+1/2b^TDb)$ with the constraints $A^Tb>=b_0$.

In [263]:
def get_weight(x, method, risk_free=None):
    n_cluster = x.shape[1]
    zeros = np.zeros(n_cluster).T
    
    if method == "GMV":
        A = np.c_[np.ones(n_cluster), np.diag(np.ones(n_cluster))]
        b = np.array([1] + [0]*n_cluster, dtype=np.float64)
    elif method == "Tangency":
        if not risk_free:
            raise ValueError("method is 'Tangency'. 'risk_free' should not be None")
        rf = risk_free.r.mean()
        A = np.c_[x.mean() - rf, np.diag([1] * n_cluster)]
        b = np.array([(x.mean() - rf).sum(), [0] * n_cluster], dtype=np.float64)
    else:
        raise ValueError("invalid method")
    
    qp = solve_qp(x.cov().to_numpy(), zeros, A, b, meq=1)
    return qp[0]   # solution

In [ ]:
def get_portfolio_return(data, timeset, n_time, with_, method,
                         market, risk_free, random_state=0):
    p_return = np.zeros(len(timeset))
    for i in range(len(timeset)):
        

In [ ]:
def expand_grid(*args):
    pass

In [ ]:
def evaluate_portfolio(data, market, risk_free, start, end,
                       with_list, n_time_list, method_list, random_state):
    pass

---

In [258]:
n_cluster = xx.shape[1]
qp = solve_qp(xx.cov().to_numpy(), np.zeros(n_cluster).T, 
              np.c_[np.ones(n_cluster), np.diag(np.ones(n_cluster))],
              np.array([1] + [0]*n_cluster, dtype=np.float64), meq=1)

In [262]:
qp[0]

array([0.14471702, 0.21681592, 0.29588222, 0.34258483])

In [245]:
xx = x.iloc[:, 2:]

In [246]:
xx.cov()

,PC1,PC2,PC3,PC4
PC1,1.757408e+00,1.886440e-16,3.941071e-17,6.497025e-17
PC2,1.886440e-16,1.173008e+00,-6.911705e-17,1.628753e-16
PC3,3.941071e-17,-6.911705e-17,8.595544e-01,2.826133e-16
PC4,6.497025e-17,1.628753e-16,2.826133e-16,7.423763e-01


In [26]:
n_cluster = 3

In [27]:
np.c_[ np.ones(3), np.diag(np.ones(n_cluster)) ]

array([[1., 1., 0., 0.],
       [1., 0., 1., 0.],
       [1., 0., 0., 1.]])

In [28]:
np.array([1] + [0]*n_cluster)

array([1, 0, 0, 0])

---

# Clustering
## `src/functions-clustering.R` $\rightarrow$ `src/clustering.py`

In [16]:
import statsmodels.api as sm
from scipy.spatial.distance import squareform, pdist
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from tqdm import tqdm

In [11]:
def time_slice(data, time_idx):
    as_list = list(time_idx)
    return stock_tbl.groupby(["code"]).nth(as_list).reset_index()

In [12]:
def time_expand(data, skip=[0,1]):
    cols = [col for col in range(data.shape[1]) if col not in skip]
    
    while len(set(data.time)) > 1:
        lagged = data.iloc[:, cols].shift(1)
        lagged.columns = [f"x{c}" for c in cols]
        
        data = pd.concat([data, lagged], axis=1)
        data = data.groupby(["code"], as_index=False).apply(lambda x: x.iloc[1:]).reset_index(drop=True)
        
        cols = np.array(cols) + len(cols)
        
    return data

In [13]:
def scale_tbl(data, skip=[0,1]):
    idx = [i for i in range(data.shape[1]) if i not in skip]
    vals = data.iloc[:, idx]
    data.iloc[:, idx] = (vals - vals.mean()) / vals.std()
    return data.reset_index(drop=True)

##### PCA에서 데이터 절반이 날아감

In [15]:
def pca_(data, skip=[0,1], threshold=.8):
    """
    데이터에 PCA(주성분분석)을 수행한다.
    
    INPUT:
      data: data frame
      skip: PCA 대상에서 제외할 열 번호 (integer vector)
      threshold: 주성분 개수 선택의 기준이 되는 변동의 비율 (0 ~ 1)
    
    RETURN:
      변수들이 주성분으로 대체된 data frame
    """
    idx = [i for i in range(data.shape[1]) if i not in skip]
    omit_na = data.iloc[:, idx].dropna()    #### 여기 dropna() 때문에 데이터 절반이 날아감
    
    pca = PCA()
    x_pc = pd.DataFrame(pca.fit_transform(omit_na))
    n_pc = np.where(pca.explained_variance_ratio_.cumsum() > threshold)[0][0] + 1
    x_pc = x_pc.iloc[:, :n_pc]
    x_pc.columns = [f"PC{c+1}" for c in x_pc.columns]
    
    df_pc = pd.concat([data.dropna().iloc[:, skip].reset_index(drop=True), x_pc], axis=1)
    return df_pc

In [54]:
def add_factors_residual(data, risk_free):
    idx_rf = [True if t in data.time.tolist() else False for t in risk_free.time]
    risk_free = risk_free.iloc[idx_rf, :]
    
    for code in tqdm(data.code.unique()):
        data.loc[data.code == code, "logret"] = data[data.code == code]["logret"].shift(-1)
        data.loc[data.code == code, "rf"] = risk_free.r.shift(-1)
    
    y = data.logret - data.rf
    x = pca_(data.drop(['logret', 'rf'], axis=1))
    
    df_pca = pd.concat([x, y], axis=1).iloc[:, 2:]
    df_pca.rename(index=str, columns={0: "y"}, inplace=True)
    fml = " + ".join([c for c in df_pca.columns if "PC" in c])
    lmfit = sm.OLS.from_formula(f"y ~ {fml}", data=df_pca).fit()
    
    yhat = lmfit.predict(x)
    res = y - yhat
    data["factors_res"] = (res - res.mean()) / res.std()
    
    n_time = data.time.unique().shape[0]
    dfs = [data.loc[data.code == code, :].iloc[:-1, :] for code in tqdm(data.code.unique())]
    data = pd.concat(dfs)
    
    return data

In [55]:
def add_market_residual(data, market, risk_free):
    idx_mk = [True if t in data.time.tolist() else False for t in market.time]
    market = market.iloc[idx_mk, :]
    
    idx_rf = [True if t in data.time.tolist() else False for t in risk_free.time]
    risk_free = risk_free.iloc[idx_rf, :]
    
    for code in tqdm(data.code.unique()):
        data.loc[data.code == code, "mk"] = market.logret
        data.loc[data.code == code, "rf"] = risk_free.r
        y = data.logret - data.rf
        x = data.mk - data.rf
    
    lmfit = sm.OLS.from_formula("y ~ x", data=data).fit()
    yhat = lmfit.predict(lmfit)
    res = y - yhat
    data["market_res"] = (res - res.mean()) / res.std()
    
    return data

In [152]:
def kmeanspp(x, k, random_state=0):
    #n = x.shape[0]
    #centers = [0] * k
    #centers[0] = np.random.randint(1, n+1)
    #
    #L2_mat = pd.DataFrame(squareform(pdist(x.iloc[:, 1:])), columns=x.index.unique(), index=x.index.unique())
    #L2_mat = L2_mat ** 2
    #
    #for i in range(1, k):
    #    weight = l2.iloc[:, centers].apply(np.min, axis=1)
    #    centers[i] = np.random.choice(range(1, n+1), p=weight/weight.sum())
    return KMeans(n_clusters=k, random_state=random_state).fit(x)

* Dunn index 대신 silhouette score를 사용
* 속도 느림

In [179]:
def get_kmeans_tbl(data, ncmin=2, ncmax=5):
    data = data.dropna()
    
    ncs = range(ncmin, nvmax)
    
    X = data.iloc[:, 2:]
    models = [kmeanspp(X, nc) for nc in ncs]
    silhouettes = np.array([silhouette_score(X, m.labels_) for m in tqdm(models)])
    best_model = models[silhouettes.argmax()]
    
    return pd.DataFrame(np.array([data.code.unique(), kmeans.labels_]).T, columns=["code", "cluster"])

In [180]:
def kmeans_with(data, with_, market, risk_free):
    if with_ == "return":
        return data.loc[:, ["code", "time", "logret"]]
    elif with_ == "market_residual":
        return add_market_residual(data, market, risk_free).loc[:, ["code", "time", "market_res"]]
    elif with_ == "factors":
        return pca_(data.drop(["logret"], axis=1))
    elif with_ == "factors_residual":
        return add_factors_residual(data, risk_free).loc[:, ["code", "time", "factors_res"]]
    else:
        raise ValueError("'with_' should be one of ['return', 'market_residual', 'factors', 'factors_residual']")

In [181]:
def integrate_return(return_, weight):
    weight = np.array(weight)
    weight = weight / weight.sum()
    return np.log(np.sum(weight * np.exp(return_)))

In [185]:
def integrate_return_apply(row):
    return integrate_return(row["logret"], row["size"])

In [227]:
def get_cluster_return(data, time_idx, with_, market, risk_free):
    cluster_df = time_slice(data, time_idx)
    cluster_df = scale_tbl(cluster_df)
    cluster_df = kmeans_with(with_, market, risk_free)
    cluster_df = time_expand(cluster_df)
    cluster_df = get_kmeans_tbl(cluster_df)
    
    data = data.merge(cluster_df, how="left", on=["code"]).loc[:, ["code", "time", "logret", "size", "cluster"]]
    data["size"] = data.size.shift(1)
    data.dropna(inplace=True)
    
    data["logret"] = data.groupby(["cluster", "time"]).apply(integrate_return_apply).reset_index(drop=True)
    data = pd.DataFrame(pd.pivot_table(data, values='logret', index=['cluster']).T.to_records()).iloc[:, 1:]
    data.columns = [f"time{c}" for c in data.columns]
    data = data.T[0]
    
    x = data[time_idx]
    y = data[time_idx[-1] + 1]
    y_time = data.index[time_idx[-1] + 1]
    
    if (x.shape[0] == len(time_idx)) and (y_time == data.time.unique()[time_idx[-1]+1]):
        return {"x": x, "y": y}
    else:
        raise ValueError()

---

---

# Main script

## `02_main.R`

In [228]:
from src.clustering import *
from src.portfolio import *

In [1]:
import pandas as pd
import numpy as np

## Load data

## Models

In [19]:
with_list = ["return", "market_residual", "factors", "factors_residual"]
n_time_list = [6, 8, 10, 12]
method_list = ["GMV", "Tangency"]

In [ ]:
start_list <- str_c(c("2002", "2005", "2008", "2011"), "-4")
end_list <- str_c(c("2005", "2008", "2011", "2014"), "-3")
valid_res_list <- list()

In [ ]:
start_list = ["2002-4", "2005-4", "2008-4", "2011-4"]
end_list = ["2005-3", "2008-3", "2011-3", "2014-3"]

validation_result = []
for i in range(4):
    st = start_list[i]
    en = end_list[i]
    valid_res_list[[i]] = evaluate_portfolio(stock_tbl, kospi, risk_free, start, end,
                                             with_list, n_time_list, method_list)